<a href="https://colab.research.google.com/github/ajitjadhav10/Personal-Projects/blob/main/News%20Summarization%20Application/T5_fine_tuning_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Summarization Application Using Fine-Tuned T5 Transformer Model

- In this project we have built a news summarization application on Streamlit. This application summarizes live news articles which are fetched using Google News RSS feed
- We have used the CNN/Daily Mail news dataset (https://huggingface.co/datasets/cnn_dailymail) for fine-tuning the T5 transformer.
- This fine-tuned model is then used to build a summarization application on Streamlit

## Code for fine-tuning T5 transformer model:

In [ ]:
import pandas as pd

In [ ]:
! pip install datasets transformers rouge-score nltk -q

     |████████████████████████████████| 451 kB 5.2 MB/s 
     |████████████████████████████████| 5.5 MB 71.3 MB/s 
     |████████████████████████████████| 115 kB 75.4 MB/s 
     |████████████████████████████████| 212 kB 82.1 MB/s 
     |████████████████████████████████| 182 kB 82.2 MB/s 
     |████████████████████████████████| 127 kB 81.0 MB/s 
     |████████████████████████████████| 7.6 MB 60.6 MB/s 


In [ ]:
pip install huggingface_hub -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Connecting hugging face account to the notebook to save the model in hugging face hub once the model is fine-tuned

from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
import datasets

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.24.0


In [ ]:
model_checkpoint = "t5-small"

In [ ]:
#Importing the CNN/Daily Mail dataset

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("cnn_dailymail",'3.0.0')
metric = load_metric("rouge")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [ ]:
#Looking at the contents of the dataset

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
#Having a look at one row from the train dataset

raw_datasets["train"][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
#Displaying the contents of the CNN/Daily Mail dataset

show_random_elements(raw_datasets["train"])

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [ ]:
#Example of how the metrics will be computed

fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
#Example of how the sentence in the dataset will be tokenized

tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
#Preparing the text before feeding it ot the tokenizer

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[21603, 10, 301, 24796, 4170, 6, 2789, 41, 18844, 61, 1636, 8929, 16023, 2213, 4173, 6324, 12591, 15, 11391, 592, 12, 3, 9, 2196, 3996, 1755, 770, 8785, 591, 11039, 770, 61, 13462, 38, 3, 88, 5050, 507, 30, 2089, 6, 68, 3, 88, 10419, 7, 8, 540, 751, 31, 17, 4061, 3, 9, 10783, 30, 376, 5, 4173, 6324, 12591, 15, 38, 8929, 16023, 16, 96, 15537, 651, 16023, 11, 8, 5197, 13, 8, 12308, 121, 304, 8, 19142, 13, 29517, 6710, 343, 7, 300, 8, 296, 6, 8, 1021, 7556, 845, 3, 88, 65, 150, 1390, 12, 9030, 17, 449, 112, 1723, 550, 30, 1006, 2948, 6, 3281, 11, 17086, 2251, 5, 96, 196, 278, 31, 17, 515, 12, 36, 80, 13, 273, 151, 113, 6, 38, 1116, 38, 79, 919, 14985, 8247, 805, 1452, 3, 9, 3805, 2100, 443, 1232, 42, 424, 1126, 976, 3, 88, 1219, 46, 3746, 2772, 49, 2283, 48, 847, 5, 96, 196, 278, 31, 17, 317, 27, 31, 195, 36, 1989, 28887, 5, 96, 634, 378, 27, 114, 2611, 33, 378, 24, 583, 81, 335, 7051, 1636, 1335, 11, 3190, 7, 11, 5677, 7, 535, 486, 14985, 6324, 12591, 15, 56, 36, 3, 179, 1

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/288 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
#Hyper-parameters used for training the model

batch_size = 12
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-t5-summarization",
    evaluation_strategy = "epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/ajitjadhav/t5-small-finetuned-t5-summarization into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.43k/231M [00:00<?, ?B/s]

Download file runs/Nov24_01-37-27_50db25be8968/1669260824.1329968/events.out.tfevents.1669260824.50db25be8968.…

Download file runs/Nov24_01-37-27_50db25be8968/1669261133.2322292/events.out.tfevents.1669261133.50db25be8968.…

Download file runs/Nov24_01-37-27_50db25be8968/events.out.tfevents.1669261133.50db25be8968.77.9:  32%|###2    …

...

Clean file runs/Nov24_01-37-27_50db25be8968/events.out.tfevents.1669260932.50db25be8968.77.3:  22%|##1       |…

Clean file runs/Nov24_01-37-27_50db25be8968/events.out.tfevents.1669260988.50db25be8968.77.5:  22%|##1       |…

Clean file training_args.bin:  29%|##8       | 1.00k/3.48k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

Using cuda_amp half precision backend


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Training the model

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: article, id, highlights. If article, id, highlights are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 287113
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 107670
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` me

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.989100,1.798088,24.382000,11.709900,20.170700,23.002100,18.999800
2,1.952700,1.781631,24.488400,11.767300,20.269800,23.123300,19.000000


Saving model checkpoint to t5-small-finetuned-t5-summarization/checkpoint-500
Configuration saved in t5-small-finetuned-t5-summarization/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-t5-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-t5-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-t5-summarization/checkpoint-500/special_tokens_map.json
tokenizer config file saved in t5-small-finetuned-t5-summarization/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-t5-summarization/special_tokens_map.json
Saving model checkpoint to t5-small-finetuned-t5-summarization/checkpoint-1000
Configuration saved in t5-small-finetuned-t5-summarization/checkpoint-1000/config.json
Model weights saved in t5-small-finetuned-t5-summarization/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-t5-summarization/checkpoint-1000/tokenizer_config

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.989100,1.798088,24.382000,11.709900,20.170700,23.002100,18.999800
2,1.952700,1.781631,24.488400,11.767300,20.269800,23.123300,19.000000
3,1.942100,1.772838,24.578200,11.840100,20.334300,23.203300,18.999700
4,1.929800,1.767682,24.566000,11.872300,20.329600,23.194300,18.999900
5,1.925600,1.761927,24.566200,11.838500,20.326500,23.201600,18.999900
6,1.905600,1.761284,24.575500,11.842400,20.303100,23.186700,18.999900


Saving model checkpoint to t5-small-finetuned-t5-summarization/checkpoint-54000
Configuration saved in t5-small-finetuned-t5-summarization/checkpoint-54000/config.json
Model weights saved in t5-small-finetuned-t5-summarization/checkpoint-54000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-t5-summarization/checkpoint-54000/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-t5-summarization/checkpoint-54000/special_tokens_map.json
Deleting older checkpoint [t5-small-finetuned-t5-summarization/checkpoint-52500] due to args.save_total_limit
Saving model checkpoint to t5-small-finetuned-t5-summarization/checkpoint-54500
Configuration saved in t5-small-finetuned-t5-summarization/checkpoint-54500/config.json
Model weights saved in t5-small-finetuned-t5-summarization/checkpoint-54500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-t5-summarization/checkpoint-54500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-

TrainOutput(global_step=107670, training_loss=1.948579710156758, metrics={'train_runtime': 13682.7451, 'train_samples_per_second': 125.901, 'train_steps_per_second': 7.869, 'total_flos': 2.33150343876182e+17, 'train_loss': 1.948579710156758, 'epoch': 6.0})

Saving the model on huggingface hub
- After saving the model on huggingface hub, it can be used by anyone from this link: https://huggingface.co/ajitjadhav/t5-small-finetuned-t5-summarization

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to t5-small-finetuned-t5-summarization
Configuration saved in t5-small-finetuned-t5-summarization/config.json
Model weights saved in t5-small-finetuned-t5-summarization/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-t5-summarization/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-t5-summarization/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/231M [00:00<?, ?B/s]

Upload file runs/Dec01_16-42-17_15788a09b431/events.out.tfevents.1669913039.15788a09b431.106.0:   8%|7        …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/ajitjadhav/t5-small-finetuned-t5-summarization
   804b6be..1a3afbd  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ajitjadhav/t5-small-finetuned-t5-summarization
   804b6be..1a3afbd  main -> main

To https://huggingface.co/ajitjadhav/t5-small-finetuned-t5-summarization
   1a3afbd..c69955f  main -> main

   1a3afbd..c69955f  main -> main



'https://huggingface.co/ajitjadhav/t5-small-finetuned-t5-summarization/commit/1a3afbd6426ac85dc4ee68941b0feac66fe8a464'

### Importing our fine-tuned model from hugging face hub for summarization (https://huggingface.co/ajitjadhav/t5-small-finetuned-t5-summarization)

In [ ]:
!pip install -qq torch

In [ ]:
!pip install -qq sentencepiece

In [ ]:
pip install transformers -q

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ajitjadhav/t5-small-finetuned-t5-summarization")

model = AutoModelForSeq2SeqLM.from_pretrained("ajitjadhav/t5-small-finetuned-t5-summarization")

In [ ]:
import torch
import json
device = torch.device('cpu')

In [ ]:
text =""" The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that."""


preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

original text preprocessed: 
 The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors."We'll be the comeback kids, all of us," he said. "We want to get our country back."The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.


Summarized text: 
 President Donald Trump predicts some states will reopen this month. The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world. "We'll be the comeback kids, all of us," he says.


Calculating Flesch score of the complete and summarized text

In [ ]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from pandas import DataFrame
import textstat

In [ ]:
complete_text=""" The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that."""

complete_text=str(complete_text)

In [ ]:
print(textstat.flesch_reading_ease(complete_text))
print(textstat.reading_time(complete_text, ms_per_char=70))

67.89
40.95


In [ ]:
summarized_text="President Donald Trump predicts some states will reopen this month. The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world."
summarized_text=str(summarized_text)

In [ ]:
print(textstat.flesch_reading_ease(summarized_text))
print(textstat.reading_time(summarized_text, ms_per_char=70))

81.63
10.71


From the above Flesch score, we can see that our model provides users with an easy to comprehend summary and also reduces reading time.